In [30]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np

from GEMS_TCO import kernels
from GEMS_TCO import data_preprocess 
from GEMS_TCO import kernels_new, kernels_reparam_space_time as kernels_reparam_space_time
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
from json import JSONEncoder

from GEMS_TCO.data_loader import load_data2
import torch
import torch.optim as optim
import time

Load monthly data

In [53]:
space: List[str] = ['4', '4']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 20
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)


df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
lat_range=[0.0, 5.0],      
lon_range=[123.0, 133.0] 
)

#days: List[str] = ['0', '31']
#days_s_e = [int(d) for d in days]
#days_list = list(range(days_s_e[0], days_s_e[1]))

Subsetting data to lat: [0.0, 5.0], lon: [123.0, 133.0]


Load daily data applying max-min ordering

In [54]:
daily_aggregated_tensors = [] 
daily_hourly_maps = []        

analysis_hour =2
k=0
for day_index in range(31):
  
    hour_start_index = day_index * 8 + k 


    hour_end_index = (day_index + 1) * 8 + k
    #hour_end_index = day_index*8 + 1
    #hour_end_index = day_index*8 + analysis_hour + k

    hour_indices = [hour_start_index, hour_end_index]
    
    # Load the data for the current day
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
        df_map, 
        hour_indices, 
        ord_mm=None,  
        dtype=torch.float 
    )
    # Append the day's data to their respective lists
    daily_aggregated_tensors.append(day_aggregated_tensor)
    daily_hourly_maps.append(day_hourly_map) 

print(daily_aggregated_tensors[0].shape)
#print(daily_hourly_maps[0])

torch.Size([8960, 4])


Hyper parameters

In [55]:
v = 0.5 # smooth
mm_cond_number = 8
nheads = 300
#nheads = 1230
#lr = 0.01
#step = 80
#gamma_par = 0.5

# --- Placeholder Global Variables ---
# 💥 REVISED: Added lr, patience, factor. Removed step, gamma_par
lr=0.1
patience = 5       # Scheduler: Epochs to wait for improvement
factor = 0.5         # Scheduler: Factor to reduce LR by (e.g., 0.5 = 50% cut)
epochs=150

In [56]:
def cal(a):
    day_indices = [0] 
    for day_idx in day_indices:  

        daily_hourly_map = daily_hourly_maps[day_idx]
        daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

        # Convert initial parameters to a list of 1-element tensors
        params_list = [
            torch.tensor([val], dtype=torch.float64, requires_grad=True, device=device_str) for val in a
        ]
        
        # 💡 CRITICAL: Concatenate the list into a single tensor
        # The full_likelihood function expects a single tensor, not a list
        params_tensor = torch.cat(params_list)

        # Assuming 'kernels_repar_space' has your 'fit_vecchia_adams' class
  
        model_instance = kernels_reparam_space_time.fit_vecchia_adams(
                smooth = v,
                input_map = daily_hourly_map,
                aggregated_data = daily_aggregated_tensor,
                nns_map = nns_map,
                mm_cond_number = mm_cond_number,
                nheads = nheads
            )
    
        # 💡 Pass the single 'params_tensor' and the correct 4-parameter spatial covariance function
        bb = model_instance.full_likelihood_avg(
            params = params_tensor, 
            input_data = daily_aggregated_tensor, 
            y = daily_aggregated_tensor[:,2], 
            covariance_function = model_instance.matern_cov_aniso_STABLE_log_reparam
        )
     
        cov_map = model_instance.cov_structure_saver(params_tensor, model_instance.matern_cov_aniso_STABLE_log_reparam)
        vecchia_nll = model_instance.vecchia_space_time_fullbatch( # Change this to your chosen Vecchia implementation
            params = params_tensor, 
            covariance_function = model_instance.matern_cov_aniso_STABLE_log_reparam, 
            cov_map = cov_map # Assuming cov_map is precomputed or computed internally
        )
 
    return bb, vecchia_nll

device_str = 'cuda' if torch.cuda.is_available() else 'cpu'
# adams and lfgs
#a= [3.2691297610712176, 0.6684659526552683, 0.4027674047362919, 0.8989261051578574, 6521.4909959990455]

#bb, nll_value = cal(a)
#print(f"Full Negative Log-Likelihood: {bb.item()*1120, nll_value.item()}")

def cc(n,nheads,mm):
    print(n**2 , nheads**2 + (n-nheads)*mm**3)
    return n**2 >nheads**2 + (n-nheads)*mm**3

cc(1000,300,8)

1000000 448400


True

# 8 hours 1120

###  vecchia

In [57]:
a = [4.286645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")

a = [4.386645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")

a = [4.186645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")


Full Negative Log-Likelihood: (29432.74468523959, 30591.837590380615)
Full Negative Log-Likelihood: (29526.910924277436, 30683.88178585764)
Full Negative Log-Likelihood: (29424.180472720167, 30585.618627548574)


### dw adams


In [58]:
a = [4.2739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")

a = [4.3739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")

a =[4.1739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")


Full Negative Log-Likelihood: (29660.142481601535, 30665.408140860818)
Full Negative Log-Likelihood: (29739.981851793724, 30721.93854034474)
Full Negative Log-Likelihood: (29667.41189446016, 30698.437968010054)
